# Data Preparation Notebook (Annotating + Filtering)

## Imports and Loading Dataset

In [1]:
import pandas as pd
import os

# Basic
pr_df = pd.read_parquet("hf://datasets/hao-li/AIDev/pull_request.parquet")
repo_df = pd.read_parquet("hf://datasets/hao-li/AIDev/repository.parquet")
user_df = pd.read_parquet("hf://datasets/hao-li/AIDev/user.parquet")

# Comments and reviews
pr_comments_df = pd.read_parquet(
    "hf://datasets/hao-li/AIDev/pr_comments.parquet")
pr_reviews_df = pd.read_parquet(
    "hf://datasets/hao-li/AIDev/pr_reviews.parquet")
pr_review_comments_df = pd.read_parquet(
    "hf://datasets/hao-li/AIDev/pr_review_comments_v2.parquet")

# Commits
pr_commits_df = pd.read_parquet(
    "hf://datasets/hao-li/AIDev/pr_commits.parquet")
pr_commit_details_df = pd.read_parquet(
    "hf://datasets/hao-li/AIDev/pr_commit_details.parquet")

# Related issues
related_issue_df = pd.read_parquet(
    "hf://datasets/hao-li/AIDev/related_issue.parquet")
issue_df = pd.read_parquet("hf://datasets/hao-li/AIDev/issue.parquet")

# Events
pr_timeline_df = pd.read_parquet(
    "hf://datasets/hao-li/AIDev/pr_timeline.parquet")

# Task type
pr_task_type_df = pd.read_parquet(
    "hf://datasets/hao-li/AIDev/pr_task_type.parquet")

# Human-PR
human_pr_df = pd.read_parquet(
    "hf://datasets/hao-li/AIDev/human_pull_request.parquet")
human_pr_task_type_df = pd.read_parquet(
    "hf://datasets/hao-li/AIDev/human_pr_task_type.parquet")

print(f"len(pr_df): {len(pr_df)}")
print(f"len(repo_df): {len(repo_df)}")
print(f"len(user_df): {len(user_df)}")

print(f"\nlen(pr_comments_df): {len(pr_comments_df)}")
print(f"len(pr_reviews_df): {len(pr_reviews_df)}")
print(f"len(pr_review_comments_df): {len(pr_review_comments_df)}")

print(f"\nlen(pr_commits_df): {len(pr_commits_df)}")
print(f"len(pr_commit_details_df): {len(pr_commit_details_df)}")


print(f"\nlen(related_issue_df): {len(related_issue_df)}")
print(f"len(issue_df): {len(issue_df)}")

print(f"\nlen(pr_timeline_df): {len(pr_timeline_df)}")

print(f"\nlen(pr_task_type_df): {len(pr_task_type_df)}")

print(f"\nlen(human_pr_df): {len(human_pr_df)}")
print(f"len(human_pr_task_type_df): {len(human_pr_task_type_df)}")

len(pr_df): 33596
len(repo_df): 2807
len(user_df): 1796

len(pr_comments_df): 39122
len(pr_reviews_df): 28875
len(pr_review_comments_df): 26868

len(pr_commits_df): 88576
len(pr_commit_details_df): 711923

len(related_issue_df): 4923
len(issue_df): 4614

len(pr_timeline_df): 325500

len(pr_task_type_df): 33596

len(human_pr_df): 6618
len(human_pr_task_type_df): 6618


## Annotating Features

New columns added to both human and agentic PR dataframes:

- accepted (bool) - merged PRs
- rejected (bool) - PRs that were closed but not merged
- turnaround_time (seconds) - number of seconds between creation and decision (merged/closed)
- days_to_close (days) - number of days between creation and decision (merged/closed)

New columns added to only agentic PR dataframes:

- related_issue (bool) - if the PR has a related issue
- has_open_related_issue (bool) - if at least one of the related issues are open
- num_commits (int) - number of commits in the PR
- num_files_changed (int) - number of files touched in the PR
- touches_test_file (bool) - whether "test" appears in any of the filenames modified by the PR
- lines_added (int) - number of lines added throughout all PR commits
- lines_deleted (int) - number of lines deleted throughout all PR commits
- total_churn (int) - total churn across all PR commits (lines added + lines deleted)
- net_churn (int) - net churn across all PR commits (lines added - lines deleted)
- num_bot_users (int) - how many bots were involved in the PR (either in comments or reviews)
- num_human_users (int) - how many humans were involved in the PR (either in comments or reviews)
- num_total_users (int) - how many users (bots & humans) were involved in the PR (either in comments or reviews)
- num_comments (int) - total number of comments (bot & human)
- num_human_comments (int) - number of human comments
- num_bot_comments (int) - number of bot comments
- num_reviews (int) - total number of reviews (bot & human)
- num_human_reviews (int) - number of human reviews
- num_bot_reviews (int) - number of bot reviews
- has_comment (bool) - if the PR has at least one comment
- has_review (bool) - if the PR has at least one review

In [2]:
def annotate_pr_features(pr_dataframe, human=False):
    """
    Annotate PR dataframe with additional features.

    Parameters:
    - pr_dataframe: DataFrame containing pull request data

    Returns:
    - Annotated PR DataFrame
    """
    # Create a copy to avoid modifying the original
    df = pr_dataframe.copy()

    # Drop columns if they already exist
    columns_to_drop = ['accepted', 'rejected', 'pending', 'turnaround_time', 'days_to_close', 'related_issue', 'has_open_related_issue', 'num_commits', 'num_files_changed', 'touches_test_file', 'lines_added', 'lines_deleted',
                       'total_churn', 'net_churn', 'num_bot_users', 'num_human_users', 'num_total_users', 'num_comments', 'num_human_comments', 'num_bot_comments', 'num_reviews', 'num_human_reviews', 'num_bot_reviews', 'has_comment', 'has_review']
    df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])

    df['accepted'] = df['merged_at'].notnull()
    df['rejected'] = (df['state'] == 'closed') & df['merged_at'].isnull()
    df['turnaround_time'] = (pd.to_datetime(
        df['closed_at']) - pd.to_datetime(df['created_at'])).dt.total_seconds()
    df['days_to_close'] = (pd.to_datetime(
        df['closed_at']) - pd.to_datetime(df['created_at'])).dt.days + 1

    if not human:
        # Related Issue
        df['related_issue'] = df['id'].isin(
            related_issue_df['pr_id'].unique())

        # Has Open Related Issue
        merged_related_issues = related_issue_df.merge(
            issue_df[['id', 'state']], left_on='issue_id', right_on='id', how='left')
        pr_open_issues = merged_related_issues[merged_related_issues['state'] == 'open']
        pr_has_open_issue = pr_open_issues['pr_id'].unique()
        df['has_open_related_issue'] = df['id'].isin(pr_has_open_issue)

        # Number of commits per PR
        pr_commit_counts = pr_commits_df.groupby(
            'pr_id').size().reset_index(name='num_commits')
        df = df.merge(pr_commit_counts, left_on='id',
                      right_on='pr_id', how='left')
        df['num_commits'] = df['num_commits'].fillna(0).astype(int)
        df = df.drop('pr_id', axis=1)

        # Number of files changed per PR
        pr_files_changed = pr_commit_details_df.groupby(
            'pr_id')['filename'].nunique().reset_index()
        pr_files_changed.columns = ['id', 'num_files_changed']
        df = df.merge(pr_files_changed, on='id', how='left')

        def touches_test_file(filenames):
            for filename in filenames:
                if filename is not None and 'test' in filename.lower():
                    return True
            return False

        pr_files = pr_commit_details_df.groupby(
            'pr_id')['filename'].apply(list).reset_index()
        pr_files['touches_test_file'] = pr_files['filename'].apply(
            touches_test_file)
        df = df.merge(pr_files[['pr_id', 'touches_test_file']],
                      left_on='id', right_on='pr_id', how='left')
        df = df.drop('pr_id', axis=1)
        df['touches_test_file'] = df['touches_test_file'].fillna(False)

        # PR Size (number of lines added and deleted across all commits)
        pr_size = pr_commit_details_df.groupby('pr_id').agg(
            {'additions': 'sum', 'deletions': 'sum'}).reset_index()
        pr_size.columns = ['id', 'lines_added', 'lines_deleted']
        df = df.merge(pr_size, on='id', how='left')

        # Net Churn & Total Churn
        df['net_churn'] = df['lines_added'] - df['lines_deleted']
        df['total_churn'] = df['lines_added'] + df['lines_deleted']

        ### Number of Human/Bot/Total Reviews/Comments/Involvement ###

        # Get unique users from comments
        comment_users = pr_comments_df.groupby('pr_id').apply(
            lambda x: list(zip(x['user'], x['user_type']))
        ).reset_index()
        comment_users.columns = ['pr_id', 'comment_users']

        # Get comment counts by type
        comment_counts = pr_comments_df.groupby('pr_id').apply(
            lambda x: pd.Series({
                'num_comments': len(x),
                'num_human_comments': sum(x['user_type'] == 'User'),
                'num_bot_comments': sum(x['user_type'] == 'Bot')
            })
        ).reset_index()

        # Get unique users from reviews
        review_users = pr_reviews_df.groupby('pr_id').apply(
            lambda x: list(zip(x['user'], x['user_type']))
        ).reset_index()
        review_users.columns = ['pr_id', 'review_users']

        # Get review counts by type
        review_counts = pr_reviews_df.groupby('pr_id').apply(
            lambda x: pd.Series({
                'num_reviews': len(x),
                'num_human_reviews': sum(x['user_type'] == 'User'),
                'num_bot_reviews': sum(x['user_type'] == 'Bot')
            })
        ).reset_index()

        # Merge and combine users
        user_interactions = comment_users.merge(
            review_users, on='pr_id', how='outer')
        user_interactions['comment_users'] = user_interactions['comment_users'].apply(
            lambda x: x if isinstance(x, list) else [])
        user_interactions['review_users'] = user_interactions['review_users'].apply(
            lambda x: x if isinstance(x, list) else [])

        # Combine all users and count unique bots and humans
        def count_user_types(row):
            all_users = row['comment_users'] + row['review_users']
            unique_users = list(set(all_users))

            num_bots = sum(
                1 for user, user_type in unique_users if user_type == 'Bot')
            num_humans = sum(
                1 for user, user_type in unique_users if user_type == 'User')
            num_total = len(unique_users)

            return pd.Series({
                'num_bot_users': num_bots,
                'num_human_users': num_humans,
                'num_total_users': num_total
            })

        user_counts = user_interactions.apply(count_user_types, axis=1)
        user_interactions = pd.concat(
            [user_interactions[['pr_id']], user_counts], axis=1)

        # Merge all counts together
        user_interactions = user_interactions.merge(
            comment_counts, on='pr_id', how='outer')
        user_interactions = user_interactions.merge(
            review_counts, on='pr_id', how='outer')

        # Merge with PR dataframe
        df = df.merge(user_interactions, left_on='id',
                      right_on='pr_id', how='left')
        df = df.drop('pr_id', axis=1)

        # Fill NaN values with 0
        for col in ['num_bot_users', 'num_human_users', 'num_total_users',
                    'num_comments', 'num_human_comments', 'num_bot_comments',
                    'num_reviews', 'num_human_reviews', 'num_bot_reviews']:
            df[col] = df[col].fillna(0).astype(int)

        # Has Comment
        df['has_comment'] = df['num_comments'] > 0

        # Has Review
        df['has_review'] = df['num_reviews'] > 0

    return df


# Apply annotation to both dataframes
pr_df = annotate_pr_features(pr_df)
human_pr_df = annotate_pr_features(human_pr_df, human=True)

/var/folders/gw/jg8766954y92j4x7kpzdhg680000gp/T/ipykernel_61964/3925933268.py:65: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['touches_test_file'] = df['touches_test_file'].fillna(False)
/var/folders/gw/jg8766954y92j4x7kpzdhg680000gp/T/ipykernel_61964/3925933268.py:80: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  comment_users = pr_comments_df.groupby('pr_id').apply(
/var/folders/gw/jg8766954y92j4x7kpzdhg680000gp/T/ipykernel_61964/3925933268.py:86: FutureWarning: DataFrameGroupB

## Filtering

We apply the following filters to the dataset:

- Restricting to repos with Apache-2.0 or MIT licences
- Closed (merged/rejected) PRs only
- At least one interaction that isn't by the author before PR decision

In [ ]:
def df_filtering(pr_dataframe):
    """
    Filter PR dataframe.

    Filtering Steps:
    1. Keep only PRs with non-author reviews or comments.
    2. Keep only closed PRs.
    3. Filter repositories with Apache-2.0 or MIT licences.
    4. Filter top 10% for specified columns.

    Parameters:
    - pr_dataframe: DataFrame containing pull request data

    Returns:
    - Filtered PR DataFrame
    """

    df = pr_dataframe.copy()

    # Get PR IDs that exist in this dataframe
    existing_pr_ids = df['id'].unique()

    # PRs with non-author reviews (filter to only PRs in this dataframe)
    relevant_reviews = pr_reviews_df[pr_reviews_df['pr_id'].isin(
        existing_pr_ids)]
    df_indexed = df.set_index('id')
    
    # Filter reviews by non-author AND created before PR closed_at
    non_author_reviews_before_close = relevant_reviews[
        (relevant_reviews['user'] != df_indexed.loc[relevant_reviews['pr_id'], 'user'].values) &
        (pd.to_datetime(relevant_reviews['submitted_at']) < pd.to_datetime(df_indexed.loc[relevant_reviews['pr_id'], 'closed_at'].values))
    ]['pr_id'].unique()

    # PRs with non-author comments (filter to only PRs in this dataframe)
    relevant_comments = pr_comments_df[pr_comments_df['pr_id'].isin(
        existing_pr_ids)]
    
    # Filter comments by non-author AND created before PR closed_at
    non_author_comments_before_close = relevant_comments[
        (relevant_comments['user'] != df_indexed.loc[relevant_comments['pr_id'], 'user'].values) &
        (pd.to_datetime(relevant_comments['created_at']) < pd.to_datetime(df_indexed.loc[relevant_comments['pr_id'], 'closed_at'].values))
    ]['pr_id'].unique()

    # Union of PRs with non-author reviews and comments
    prs_with_non_author_interaction = set(
        non_author_reviews_before_close) | set(non_author_comments_before_close)
    
    # Keep only PRs with non-author review/comment
    df = df[df['id'].isin(prs_with_non_author_interaction)]
    df = df[df['state'] == 'closed']  # Only closed PRs

    # Filter repositories with Apache-2.0 or MIT licenses
    apache_mit_repos = repo_df[repo_df['license'].isin(
        ['Apache-2.0', 'MIT'])]['id'].unique()

    # Keep only PRs from those repositories
    df = df[df['repo_id'].isin(apache_mit_repos)]

    return df


# Filter dataframe
pr_df = df_filtering(pr_df)

## Filter Other Tabled for Selected PRs

In [4]:
# Get the list of PR IDs from pr_df
filtered_pr_ids = pr_df['id'].unique()

# Filter all PR-related dataframes
pr_comments_df = pr_comments_df[pr_comments_df['pr_id'].isin(filtered_pr_ids)]
pr_reviews_df = pr_reviews_df[pr_reviews_df['pr_id'].isin(filtered_pr_ids)]
# pr_review_comments_df = pr_review_comments_df[pr_review_comments_df['pull_request_url'].str.extract(r'/(\d+)$')[0].astype(int).isin(filtered_pr_ids)]

filtered_review_ids = pr_reviews_df[pr_reviews_df['pr_id'].isin(filtered_pr_ids)]['id'].unique()
pr_review_comments_df = pr_review_comments_df[pr_review_comments_df['pull_request_review_id'].isin(filtered_review_ids)]


pr_commits_df = pr_commits_df[pr_commits_df['pr_id'].isin(filtered_pr_ids)]
pr_commit_details_df = pr_commit_details_df[pr_commit_details_df['pr_id'].isin(filtered_pr_ids)]
pr_timeline_df = pr_timeline_df[pr_timeline_df['pr_id'].isin(filtered_pr_ids)]
pr_task_type_df = pr_task_type_df[pr_task_type_df['id'].isin(filtered_pr_ids)]
related_issue_df = related_issue_df[related_issue_df['pr_id'].isin(filtered_pr_ids)]

# Get unique repo IDs and user IDs from filtered PRs
filtered_repo_ids = pr_df['repo_id'].unique()
filtered_user_ids = pr_df['user_id'].unique()

# Filter repo and user dataframes
repo_df = repo_df[repo_df['id'].isin(filtered_repo_ids)]
user_df = user_df[user_df['id'].isin(filtered_user_ids)]

# Get unique issue IDs from related_issue_df
filtered_issue_ids = related_issue_df['issue_id'].unique()
issue_df = issue_df[issue_df['id'].isin(filtered_issue_ids)]

print(f"Filtered PR count: {len(pr_df)}")
print(f"Filtered pr_comments_df: {len(pr_comments_df)}")
print(f"Filtered pr_reviews_df: {len(pr_reviews_df)}")
print(f"Filtered pr_review_comments_df: {len(pr_review_comments_df)}")
print(f"Filtered pr_commits_df: {len(pr_commits_df)}")
print(f"Filtered pr_commit_details_df: {len(pr_commit_details_df)}")
print(f"Filtered pr_timeline_df: {len(pr_timeline_df)}")
print(f"Filtered pr_task_type_df: {len(pr_task_type_df)}")
print(f"Filtered related_issue_df: {len(related_issue_df)}")
print(f"Filtered human_pr_df: {len(human_pr_df)}")
print(f"Filtered human_pr_task_type_df: {len(human_pr_task_type_df)}")
print(f"Filtered repo_df: {len(repo_df)}")
print(f"Filtered user_df: {len(user_df)}")
print(f"Filtered issue_df: {len(issue_df)}")

Filtered PR count: 7156
Filtered pr_comments_df: 21243
Filtered pr_reviews_df: 17746
Filtered pr_review_comments_df: 15316
Filtered pr_commits_df: 32679
Filtered pr_commit_details_df: 235683
Filtered pr_timeline_df: 124241
Filtered pr_task_type_df: 7156
Filtered related_issue_df: 2296
Filtered human_pr_df: 6618
Filtered human_pr_task_type_df: 6618
Filtered repo_df: 1054
Filtered user_df: 571
Filtered issue_df: 2207


## Save Filtered Dataset to CSV

In [5]:
def save_dataframes_to_csv(subdirectory):
    # Check if ./data/{subdirectory} directory exists
    data_dir = "./data"
    sub_dir = os.path.join(data_dir, subdirectory)

    # Create the directory if it doesn't exist
    if not os.path.exists(sub_dir):
        os.makedirs(sub_dir)

    # Save all dataframes to CSV (will overwrite existing files)
    pr_df.to_csv(os.path.join(sub_dir, "pull_request.csv"), index=False)
    repo_df.to_csv(os.path.join(sub_dir, "repository.csv"), index=False)
    user_df.to_csv(os.path.join(sub_dir, "user.csv"), index=False)
    pr_comments_df.to_csv(os.path.join(
        sub_dir, "pr_comments.csv"), index=False)
    pr_reviews_df.to_csv(os.path.join(
        sub_dir, "pr_reviews.csv"), index=False)
    pr_review_comments_df.to_csv(os.path.join(
        sub_dir, "pr_review_comments.csv"), index=False)
    pr_commits_df.to_csv(os.path.join(
        sub_dir, "pr_commits.csv"), index=False)
    pr_commit_details_df.to_csv(os.path.join(
        sub_dir, "pr_commit_details.csv"), index=False)
    related_issue_df.to_csv(os.path.join(
        sub_dir, "related_issue.csv"), index=False)
    issue_df.to_csv(os.path.join(sub_dir, "issue.csv"), index=False)
    pr_timeline_df.to_csv(os.path.join(
        sub_dir, "pr_timeline.csv"), index=False)
    pr_task_type_df.to_csv(os.path.join(
        sub_dir, "pr_task_type.csv"), index=False)
    human_pr_df.to_csv(os.path.join(
        sub_dir, "human_pull_request.csv"), index=False)
    human_pr_task_type_df.to_csv(os.path.join(
        sub_dir, "human_pr_task_type.csv"), index=False)


save_dataframes_to_csv("filtered")

In [6]:
test_pr_df = pd.read_csv('data/filtered/pull_request.csv')
print(f"Filtered PR count: {len(test_pr_df)}")

Filtered PR count: 7156
